In [3]:
#default_exp lit_model

# LitModel
> Lit model API

In [1]:
%load_ext autoreload
%autoreload 2

# LIT

In [2]:
#export
from pytorch_lightning.core.lightning import LightningModule
import torch
from datetime import datetime, timedelta
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger
import os
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

import os.path as osp

In [44]:
#export
DEFAULT_HPARAMS = {
    "num_workers": 12,
    "image_size": 64,
    "lr": 2e-3,
    "dropout": 0.2,
    "max_epochs": 100,
    "init_lr": 8e-5,
    "num_training_steps": 2000,
}

class LitModel(LightningModule):
    def __init__(self, model, num_classes=None,hparams=DEFAULT_HPARAMS):
        super().__init__()
        if isinstance(model, str):
            assert num_classes is not None, 'num_classes cannot be none with model={}'.format(model)
            self.model = model_factory(model, num_classes)
        elif isinstance(model, torch.nn.Module):
            self.model = model

        self.loss_fn = FocalLoss()
        self._hparams = hparams


    def get_linear_schedule_with_warmup(self,
        optimizer, num_warmup_steps, num_training_steps, init_lr=5e-4, last_epoch=-1
    ):  
        from torch.optim.lr_scheduler import LambdaLR
        def lr_lambda(current_step: int):
            if current_step < num_warmup_steps:
                return float(current_step) / float(max(1, num_warmup_steps)) + init_lr
            return max(
                0.0,
                float(num_training_steps - current_step)
                / float(max(1, num_training_steps - num_warmup_steps)),
            )

        return LambdaLR(optimizer, lr_lambda, last_epoch)
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self._hparams["lr"])

        scheduler = {
            "scheduler": self.get_linear_schedule_with_warmup(
                optimizer,
                self._hparams["num_training_steps"] * 0.15,
                self._hparams["num_training_steps"],
                self._hparams["init_lr"],
            ),
            "interval": "step",  # or 'epoch'
            "frequency": 1,
        }

        return [optimizer], [scheduler]

    def forward(self, x):
        return self.model(x)

    def predict_step(self, batch, batch_idx):
        x = batch
        logits = self(x)[0]
        scores = logits.sigmoid()
        # return dict(scores=scores)
        return scores

    def validation_step(self, batch, batch_idx):
        x, y = batch[:2]
        logits = self(x)[0]
        loss = self.loss_fn(logits, y.view(len(logits),))

        preds = logits.sigmoid().argmax(1)
        accs = (y == preds).float().mean()


        self.log("val_loss", loss, rank_zero_only=True,
                    on_step=False, on_epoch=True)
        self.log("val_acc", accs, rank_zero_only=True,
                    on_step=False, on_epoch=True)

        return loss

    def training_step(self, batch, batch_idx):
        x, y = batch[:2]
        logits = self(x)[0]
        loss = self.loss_fn(logits, y.view(len(logits),))

        preds = logits.sigmoid().argmax(1)
        accs = (y == preds).float().mean()
        
        self.log("training_loss", loss, prog_bar=True, rank_zero_only=True)
        self.log("training_accuracy", accs, prog_bar=True, rank_zero_only=True)
        return loss


def get_trainer(exp_name, gpus=1, max_epochs=40,
                distributed=False, trainer_strategy='ddp',
                monitor=dict(metric="val_loss", mode="min"), save_every_n_epochs=3, 
                ):


    now = datetime.now() + timedelta(hours=7)
    root_log_dir = osp.join(
            "lightning_logs", exp_name, now.strftime(
                "%b%d-%H:%M:%S")
        )
    filename="{epoch}-{"+monitor["metric"]+":.2f}"

    callback_ckpt = ModelCheckpoint(
        dirpath=osp.join(root_log_dir, "ckpts"),
        monitor=monitor['metric'],
        filename=filename,
        mode=monitor['mode'],
        save_last=True,
        every_n_epochs=save_every_n_epochs,
    )

    callback_tqdm = TQDMProgressBar(refresh_rate=5)
    callback_lrmornitor = LearningRateMonitor(logging_interval="step")
    plt_logger = TensorBoardLogger(
        osp.join(root_log_dir, "tb_logs"), version=now.strftime("%b%d-%H:%M:%S")
    )

    trainer = Trainer(
        gpus=gpus,
        max_epochs=max_epochs,
        strategy=trainer_strategy,# if distributed else 'dp',
        callbacks=[callback_ckpt, callback_tqdm, callback_lrmornitor],
        logger=plt_logger,
    )
    return trainer

# Mnist example 

In [45]:
#hide
import os

import torch
from pytorch_lightning import LightningModule, Trainer
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import MNIST

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64

In [46]:
# to_rgb_lambda = 

In [47]:
# Init DataLoader from MNIST Dataset
train_ds = MNIST(PATH_DATASETS, train=True, download=True, transform=transforms.Compose(
[    transforms.Lambda(lambda x: x.convert('RGB')),
    transforms.ToTensor(),]
))
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

In [48]:
from lit_classifier import base_model

In [49]:
# x = train_ds[0]

In [50]:
# x[0].shape

In [51]:
model = base_model.model_factory('mobilenetv2_100', 10)

2022-05-17 11:26:20.534 | INFO     | lit_classifier.base_model:model_factory:91 - Init model mobilenetv2_100


In [52]:
trainer = get_trainer('test', gpus=None, trainer_strategy='dp', distributed=False)

MisconfigurationException: `Trainer(strategy='ddp')` or `Trainer(accelerator='ddp')` is not compatible with an interactive environment. Run your code as a script, or choose one of the compatible strategies: Trainer(strategy=None|dp|tpu_spawn). In case you are spawning processes yourself, make sure to include the Trainer creation inside the worker function.

In [53]:
# model(x)[0].shape